# 동국대학교 규정 텍스트 추출 노트북

`dongguk_rule/` 폴더 아래에 있는 모든 한글(`.hwp`) 문서에서 본문 텍스트를 추출하여 DataFrame으로 정리합니다. 추출 결과는 필요에 따라 CSV 등으로 저장할 수 있습니다.

In [14]:
import sys
import re
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple
from zipfile import BadZipFile, ZipFile
import xml.etree.ElementTree as ET

import pandas as pd

try:
    import olefile  # type: ignore
except ImportError:  # pragma: no cover
    olefile = None


In [ ]:
RULE_ROOT = Path('dongguk_rule')
OUTPUT_PATH = Path('./data/dongguk_rule_texts.csv')


In [16]:
def list_hwp_files(root: Path) -> List[Path]:
    return sorted(path for path in root.rglob('*.hwp') if path.is_file())


def extract_text_from_zip_hwp(path: Path) -> Optional[str]:
    try:
        with ZipFile(path) as zf:
            section_names = sorted(name for name in zf.namelist() if name.startswith('BodyText/Section'))
            if not section_names:
                return None
            paragraphs: List[str] = []
            for section_name in section_names:
                with zf.open(section_name) as section_file:
                    xml_data = section_file.read()
                try:
                    root = ET.fromstring(xml_data)
                except ET.ParseError:
                    continue
                texts: List[str] = []
                for tag in root.iter():
                    if tag.tag.endswith('txt') and tag.text:
                        texts.append(tag.text)
                if texts:
                    paragraphs.append(''.join(texts))
            if not paragraphs:
                return None
            return ''.join(paragraphs)
    except BadZipFile:
        return None


def extract_text_from_ole_hwp(path: Path) -> Optional[str]:
    if olefile is None:
        return None
    try:
        with olefile.OleFileIO(path) as ole:
            if ole.exists('PrvText'):
                stream = ole.openstream('PrvText')
                data = stream.read()
                if data:
                    return data.decode('utf-16-le', errors='ignore')
    except OSError:
        return None
    return None


def extract_text_from_hwp(path: Path) -> Tuple[str, Optional[str], List[str]]:
    failures: List[str] = []
    text = extract_text_from_zip_hwp(path)
    if text:
        return 'zip', text, failures

    text = extract_text_from_ole_hwp(path)
    if text:
        return 'ole', text, failures

    if olefile is None:
        failures.append('olefile_not_installed')
    failures.append('unsupported_format')
    return 'unknown', None, failures


def summarise_relative_path(path: Path, root: Path) -> Tuple[str, str]:
    rel_path = path.relative_to(root)
    parent = str(rel_path.parent)
    return parent, rel_path.name


def clean_text(text: str) -> str:
    return re.sub(r'\s+', ' ', text).strip()



In [17]:
hwp_paths = list_hwp_files(RULE_ROOT)
print(f"총 {len(hwp_paths)}개의 HWP 파일 발견")
if olefile is None:
    print('⚠️ olefile 라이브러리가 설치되어 있지 않습니다. 구형 HWP 파일은 텍스트를 추출하지 못할 수 있습니다.')

records: List[Dict[str, object]] = []
failure_details: List[Dict[str, object]] = []

for idx, path in enumerate(hwp_paths, start=1):
    method, text, failures = extract_text_from_hwp(path)
    rel_dir, filename = summarise_relative_path(path, RULE_ROOT)

    if text:
        cleaned = clean_text(text)
    else:
        cleaned = ''

    record = {
        'relative_dir': rel_dir,
        'filename': filename,
        'absolute_path': str(path.resolve()),
        'method': method,
        'text': cleaned,
    }
    records.append(record)

    if failures:
        failure_details.append({
            'path': str(path),
            'method': method,
            'issues': ';'.join(failures),
        })

    if idx % 25 == 0:
        print(f"처리 진행률: {idx}/{len(hwp_paths)}")

rule_df = pd.DataFrame(records)
print(rule_df.head())

if failure_details:
    failure_df = pd.DataFrame(failure_details)
    display(failure_df.head())
else:
    failure_df = pd.DataFrame(columns=['path', 'method', 'issues'])


총 512개의 HWP 파일 발견
처리 진행률: 25/512
처리 진행률: 50/512
처리 진행률: 75/512
처리 진행률: 100/512
처리 진행률: 125/512
처리 진행률: 150/512
처리 진행률: 175/512
처리 진행률: 200/512
처리 진행률: 225/512
처리 진행률: 250/512
처리 진행률: 275/512
처리 진행률: 300/512
처리 진행률: 325/512
처리 진행률: 350/512
처리 진행률: 375/512
처리 진행률: 400/512
처리 진행률: 425/512
처리 진행률: 450/512
처리 진행률: 475/512
처리 진행률: 500/512
         relative_dir                                          filename  \
0  제1편_학교법인             1-0-1. 학교법인 동국대학교 정관(2025.08.05.).hwp   
1  제1편_학교법인            1-0-10. 건학위원회 자문단 운영규정(2022.7.12.).hwp   
2  제1편_학교법인                           1-0-11. 법인직원인사위원회규정.hwp   
3  제1편_학교법인  1-0-12. 학교법인 동국대학교 임원 및 교직원 행동강령(2023.12.1.).hwp   
4  제1편_학교법인                     1-0-2. 정관시행세칙(2025.08.05).hwp   

                                       absolute_path method  \
0  /Users/172mac/Desktop/대학/3학년2학기/오프...    ole   
1  /Users/172mac/Desktop/대학/3학년2학기/오프...    ole   
2  /Users/172mac/Desktop/

,path,method,issues
0,dongguk_rule/제3편_행정/제4장_교무행정/의...,unknown,unsupported_format


In [18]:
rule_df.drop(columns=['absolute_path', 'method'], inplace=True)

In [19]:
# 필요하다면 추출 결과를 CSV로 저장합니다.
rule_df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')
print(f"저장 완료: {OUTPUT_PATH.resolve()}")


저장 완료: /Users/172mac/Desktop/대학/3학년2학기/오픈소스소프트웨어프로젝트/team_project/dongguk_rule_texts.csv
